# Human Activity Recognition – Assignment 1  
### 5ARE0, Academic Year 2025–2026  

**Group 13**  

**Contributors:**  
- Stan Lamerikx
- Simon Lammertink
- Philip Offermans

---

## Import Modules

Below all relevant packages which are used throughout the notebook are imported. The required packages are the same as the ones used throughout the course 5ARE0. When the setup is properly carried out according to the README.md file, all of these packages should be present

In [233]:
import zipfile
import os
import glob
import math
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz

from pathlib import Path
from typing import Iterable, Dict, List, Optional, Tuple, Union
from __future__ import annotations
from typing import Dict, List
from scipy.signal import butter, filtfilt
from collections import Counter
from scipy.optimize import linear_sum_assignment

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler

## Load Data

Below, the zip_files in the data folder are extracted. For each of the actions, a seperate directory containing the data from the sensor logger app is created. This leads to four folders:
- walking
- running 
- climbing stairs
- sitting down + standing up (these actions are combined in one recording, to be split later)

Of each of these folders, three instances exist, one for each person in the group. The folders are noted with numbers 1, 2 and 3 to preserve anonimity

In [234]:
zip_folder = "./data"   #Find data folder

data_dirs = []
# Loop through all files in the folder
for file in os.listdir(zip_folder):
    if file.endswith(".zip"):
        zip_path = os.path.join(zip_folder, file)
        extract_dir = os.path.splitext(zip_path)[0]  # remove .zip
        
        # Add folder to list of folders with data
        data_dirs.append(extract_dir)

        # Extract
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)

        print(f"Extracted {zip_path} to {extract_dir}")


Extracted ./data\climbingStairs_1.zip to ./data\climbingStairs_1
Extracted ./data\climbingStairs_2.zip to ./data\climbingStairs_2
Extracted ./data\climbingStairs_3.zip to ./data\climbingStairs_3
Extracted ./data\running_1.zip to ./data\running_1
Extracted ./data\running_2.zip to ./data\running_2
Extracted ./data\running_3.zip to ./data\running_3
Extracted ./data\sittingDown+StandingUp_1.zip to ./data\sittingDown+StandingUp_1
Extracted ./data\sittingDown+StandingUp_2.zip to ./data\sittingDown+StandingUp_2
Extracted ./data\sittingDown+StandingUp_3.zip to ./data\sittingDown+StandingUp_3
Extracted ./data\walking_1.zip to ./data\walking_1
Extracted ./data\walking_2.zip to ./data\walking_2
Extracted ./data\walking_3.zip to ./data\walking_3


The data provided by the sensor logger app is presented in a comma-seperated value file (.csv). The function below converts the files into a dataframe. In addition, the column for 'time' is dropped, since we will be using the column 'seconds_elapsed' instead as a time indication. Finally, as a check, missing values are interpolated in this function; though in practice no missing values happen to be present in the dataset.

In [235]:
def _csv_to_df(csv_path: Path) -> pd.DataFrame:
    """
    Read CSV and return as DataFrame.
    Remove the column "time"
    Check for missing values and interpolate

    """

    # Convert to dataframe
    df = pd.read_csv(csv_path)

    # Remove time column 
    if "time" in df.columns:
        df = df.drop(columns=["time"])

    # Check and interpolate missing values
    if df.isna().any().any():
        print("MISSING VALUE")
        df = df.interpolate(method="linear").dropna()
    return df

## Initialize data and cut to desired timeframe

As can also be read in the documentation report, to ensure easy data pre-processing, the data measurement started at the latest 25 seconds after beginning the recording and ended at the earliest 295 seconds after starting the recording. This means that by consistently cutting between these known points, a dataframe containing a continuous data set can be created.

To achieve this, a function is created below to cut dataframes between an input start and end time:

In [236]:
def _cut(df: pd.DataFrame, t_min: float, t_max: float) -> pd.DataFrame:

    """
    Limit dataframes to selected times
    Reset beginning to t = 0s

        Parameters: 
                t_min: start time
                t_max: end time

    """
    cut = df[(df["seconds_elapsed"] >= t_min) & (df["seconds_elapsed"] <= t_max)].copy()
    cut.sort_values("seconds_elapsed", inplace=True, kind="stable")
    cut.reset_index(drop=True, inplace=True)
    cut["seconds_elapsed"] = cut["seconds_elapsed"] - cut["seconds_elapsed"].iloc[0]

    return cut


To correctly extract only the data which we want to use, a list of strings is defined for both the sensors and actions which includes all desired data. This makes sure that, for example, the uncalibrated date is not also present in the model.

In [237]:
#Define relevant sensor data
SENSORS: list[str] = [
    "Accelerometer",
    "Gyroscope",
    "Gravity",
]

#Define list of actions
ACTIONS: list[str] = [
    "climbingStairs",
    "running",
    "sittingDown+StandingUp",
    "walking",
]

Finally, the function below uses the lists of sensors and action, as well as the function used to cut the data to create data frames which are ready for pre-processing. Notice in the code that a slightly different approach is taken for the sitting down and standing up data, since these are present in a single recording, and should thus be cut accordingly.

In [238]:
def load_data(sensors: list[str], actions: list[str], location: str = "./data/{action}*/") -> dict:
    """
    Limit data to given lists of sensors and actions.
    Cut all resulting data between relevant times.

    """
    # Create empty dict to store data
    cut_data = {}

    # Go trough all actions
    for action in actions:
        # find all recordings for an action
        rec_dirs = [d for d in glob.glob(location.format(action=action))]

        # Set the cut moment for the sitting down and standing up action
        # This end time is later, given that actions are combined into single recording
        # For more details regarding these times, check recording methodology in report
        if action == "sittingDown+StandingUp":
            t_min, t_max = 25.0, 565.1

        # Set begin and end time for all other actions
        # Times correspond to recording methodology discussed in report
        else:
            t_min, t_max = 25.0, 295.1

        
        cut_data[action] = {}

        # Go trough all recordings
        for rec in rec_dirs:
            # Go trough all sensors
            for sensor in sensors:
                # Create location of csv
                csv_path = f"{rec}{sensor}.csv"

                # Use function created before to convert to dataframe
                df = _csv_to_df(csv_path)

                # Use function before to cut dataframe to appropriate begin and end time
                df_cut = _cut(df, t_min, t_max)
                
                # Initialize list if it doesn't exist
                if sensor not in cut_data[action]:
                    cut_data[action][sensor] = [df_cut]
                else:
                    cut_data[action][sensor].append(df_cut) 
    return cut_data

data_raw = load_data(sensors=SENSORS, actions=ACTIONS, location="./data/{action}*/")

## Filter / noise reduction

Given the data initialized in the previous step, some initial pre-processing can be performed. For these purposes, a low pass filter will be applied to filter out higher-frequency noise. Continuing, a moving average filter can be applied as a form of data aggregation.

Below, two general functions are given for a low pass filter and a moving average filter. These are in correspondance with the methodology supplied in the instructions for 5ARE0.

In [ ]:
def low_pass_filter(series, cutoff_hz: float, fs: float, order: int = 4):
    """
    filter to cut off frequencies above a certain threshold

        Parameters:
                cutoff_hz: cutoff frequency (Hz)
                fs: sample rate (Hz)

    """
    nyquist = 0.5 * fs
    normal_cutoff = cutoff_hz / nyquist
    b, a = butter(order, normal_cutoff, btype="low", analog=False)
    return filtfilt(b, a, series.to_numpy())

In [ ]:
def moving_average(series, window_size: int = 5):
    """"
    Compute a centered moving average of input data
     
    """

    return series.rolling(window=window_size, center=True, min_periods=1).mean()

To properly apply the filters to all data, a seperate function is written which iterates over:

- all actions in the 'action list'
- all sensors in the 'sensor list'
- all axes in the data (x, y and z)

The function then applies the low pass and moving average filter to each.

This function is immediately applied unto the data, with the input sampling rate as 100 Hz (in correspondance with the metadata) and a cutoff frequency of 20 Hz (experimentally decided).

In [246]:
def process_low_pass_filter(data: dict, fs: float, cutoff: float) -> dict:
    """
    Apply low pass filter

    """
    filt_data = {}
    for action, sensors in data.items():
        filt_data[action] = {}
        for sensor, recordings in sensors.items():
            filt_data[action][sensor] = []
            for df in recordings:
                df_filt = df.copy()
                for axis in ["x", "y", "z"]:
                    df_filt[axis] = low_pass_filter(df[axis], cutoff_hz=cutoff, fs=fs)
                filt_data[action][sensor].append(df_filt)
    return filt_data

def process_moving_average(data: dict) -> dict:
    """
    Apply moving average
    
    """
    filt_data = {}
    for action, sensors in data.items():
        filt_data[action] = {}
        for sensor, recordings in sensors.items():
            filt_data[action][sensor] = []
            for df in recordings:
                df_filt = df.copy()
                for axis in ["x", "y", "z"]:
                    df_filt[axis] = moving_average(df[axis], window_size=5)
                filt_data[action][sensor].append(df_filt)
    return filt_data

data = process_moving_average(process_low_pass_filter(data_raw,100,20))

## Visualize data

In [201]:
# === Dash visualizer with smoothing =========================================
# Expects: data[action][sensor] -> list[pd.DataFrame] with 'seconds_elapsed' + x/y/z numeric cols

import pandas as pd
import numpy as np
from typing import List
from dash import Dash, dcc, html, Input, Output, State, no_update
import plotly.graph_objs as go

# Optional: SciPy for Butterworth + Savitzky–Golay
try:
    from scipy.signal import butter, filtfilt, savgol_filter
    SCIPY_OK = True
except Exception:
    SCIPY_OK = False

# ---------- Helpers ----------
def list_actions(data: dict) -> List[str]:
    return sorted([a for a in data.keys() if isinstance(data[a], dict)])

def list_sensors_for_action(data: dict, action: str) -> List[str]:
    if action not in data or not isinstance(data[action], dict):
        return []
    sensors = []
    for s, dfs in data[action].items():
        valid = any(isinstance(df, pd.DataFrame) and not df.empty for df in (dfs or []))
        if valid:
            sensors.append(s)
    return sorted(sensors)

def list_recording_indices(data: dict, action: str, sensor: str) -> List[int]:
    if action not in data or sensor not in data[action]:
        return []
    return [i for i, df in enumerate(data[action][sensor]) if isinstance(df, pd.DataFrame) and not df.empty]

def available_axes(df: pd.DataFrame) -> List[str]:
    axes = [c for c in ["x", "y", "z"] if c in df.columns]
    if axes:
        return axes
    return [c for c in df.select_dtypes("number").columns if c != "seconds_elapsed"]

def infer_fs(df: pd.DataFrame) -> float | None:
    if "seconds_elapsed" not in df.columns:
        return None
    t = df["seconds_elapsed"].to_numpy()
    dt = np.median(np.diff(t))
    if not np.isfinite(dt) or dt <= 0:
        return None
    return 1.0 / dt

def moving_average(x: np.ndarray, window: int) -> np.ndarray:
    if window <= 1:
        return x
    # symmetric centered window via convolution; pad to keep length
    pad = window // 2
    xpad = np.pad(x, (pad, pad), mode="edge")
    kernel = np.ones(window) / window
    y = np.convolve(xpad, kernel, mode="valid")
    return y

def apply_smoothing(df: pd.DataFrame, axes: List[str], method: str, params: dict) -> pd.DataFrame:
    """
    Returns a new DataFrame with same columns; selected axes replaced or added with smoothed data.
    If params.get("replace") is False, smoothed series are added as '<axis>_sm'.
    Supported methods: none, ma, sg, lp, hp
    """
    out = df.copy()
    if method == "none" or not axes:
        return out

    replace = bool(params.get("replace", False))
    # Downsample occurs outside; we smooth current df slice
    if method == "ma":
        window = int(params.get("ma_window", 5))
        window = max(1, window | 1)  # force odd
        for a in axes:
            if a in out.columns:
                y = moving_average(out[a].to_numpy(), window)
                out[a if replace else f"{a}_sm"] = y

    elif method == "sg" and SCIPY_OK:
        window = int(params.get("sg_window", 7))
        poly = int(params.get("sg_poly", 3))
        window = max(poly + 2 | 1, window | 1)  # ensure odd & >= poly+2
        for a in axes:
            if a in out.columns and len(out[a]) >= window:
                y = savgol_filter(out[a].to_numpy(), window_length=window, polyorder=poly, mode="interp")
                out[a if replace else f"{a}_sm"] = y

    elif method in ("lp", "hp") and SCIPY_OK:
        fs = infer_fs(out)
        if not fs:
            return out
        cutoff = float(params.get("bw_cutoff", min(fs * 0.45, fs/2 - 1e-3)))
        order = int(params.get("bw_order", 4))
        nyq = 0.5 * fs
        wc = np.clip(cutoff / nyq, 1e-6, 0.999999)
        btype = "low" if method == "lp" else "high"
        b, a = butter(order, wc, btype=btype)
        for a_col in axes:
            if a_col in out.columns and len(out[a_col]) > order * 3:
                y = filtfilt(b, a, out[a_col].to_numpy(), method="gust")
                out[a_col if replace else f"{a_col}_sm"] = y
    # If SciPy missing for sg/lp/hp, we silently leave data unchanged (raw or MA still works)
    return out

# ---------- App ----------
app = Dash(__name__)
app.title = "Recording Visualizer"

_actions = list_actions(data)
_sensors = list_sensors_for_action(data, _actions[0]) if _actions else []
_recs    = list_recording_indices(data, _actions[0], _sensors[0]) if (_actions and _sensors) else []
_rec_val = _recs[0] if _recs else None

app.layout = html.Div(
    style={"maxWidth": "1180px", "margin": "0 auto", "fontFamily": "Inter, system-ui, sans-serif"},
    children=[
        html.H2("Recordings viewer", style={"marginTop": "16px"}),

        # Row 1: dataset selectors
        html.Div(
            style={"display": "grid", "gridTemplateColumns": "1fr 1fr 1fr", "gap": "12px"},
            children=[
                html.Div([
                    html.Label("Action"),
                    dcc.Dropdown(
                        id="dd-action",
                        options=[{"label": a, "value": a} for a in _actions],
                        value=_actions[0] if _actions else None,
                        clearable=False,
                    )
                ]),
                html.Div([
                    html.Label("Sensor"),
                    dcc.Dropdown(
                        id="dd-sensor",
                        options=[{"label": s, "value": s} for s in _sensors],
                        value=_sensors[0] if _sensors else None,
                        clearable=False,
                    )
                ]),
                html.Div([
                    html.Label("Recording index"),
                    dcc.Dropdown(
                        id="dd-rec",
                        options=[{"label": str(i), "value": i} for i in _recs],
                        value=_rec_val,
                        clearable=False,
                    )
                ]),
            ],
        ),

        # Row 2: plot options
        html.Div(
            style={"display": "grid", "gridTemplateColumns": "1fr 1fr 1fr", "gap": "12px", "marginTop": "12px"},
            children=[
                html.Div([
                    html.Label("Axes to plot"),
                    dcc.Checklist(
                        id="cl-axes",
                        options=[],  # filled dynamically
                        value=[],
                        inputStyle={"marginRight": "6px", "marginLeft": "12px"},
                        inline=True,
                    ),
                ]),
                html.Div([
                    html.Label("Downsample (every Nth point)"),
                    dcc.Slider(
                        id="sl-downsample",
                        min=1, max=20, step=1, value=1,
                        marks={1: "1x", 5: "5x", 10: "10x", 20: "20x"},
                        tooltip={"placement":"bottom"}
                    ),
                ]),
                html.Div([
                    html.Label("Show mode"),
                    dcc.RadioItems(
                        id="ri-showmode",
                        options=[
                            {"label": "Raw only", "value": "raw"},
                            {"label": "Smoothed only", "value": "sm"},
                            {"label": "Overlay", "value": "overlay"},
                        ],
                        value="raw",
                        inline=True,
                    ),
                ]),
            ]
        ),

        # Row 3: smoothing controls
        html.Div(
            style={"display": "grid", "gridTemplateColumns": "1.2fr 1fr 1fr 1fr 1fr", "gap": "12px", "marginTop": "12px"},
            children=[
                html.Div([
                    html.Label("Smoothing method"),
                    dcc.Dropdown(
                        id="dd-smooth",
                        options=[
                            {"label": "None", "value": "none"},
                            {"label": "Moving Average", "value": "ma"},
                            {"label": "Savitzky–Golay", "value": "sg"},
                            {"label": "Butterworth Low-pass", "value": "lp"},
                            {"label": "Butterworth High-pass", "value": "hp"},
                        ],
                        value="none",
                        clearable=False,
                    )
                ]),
                html.Div([
                    html.Label("MA / SG window"),
                    dcc.Input(id="in-ma-sg-window", type="number", value=7, min=1, step=2, style={"width":"100%"}),
                ]),
                html.Div([
                    html.Label("SG polyorder"),
                    dcc.Input(id="in-sg-poly", type="number", value=3, min=1, step=1, style={"width":"100%"}),
                ]),
                html.Div([
                    html.Label("BW cutoff (Hz)"),
                    dcc.Input(id="in-bw-cutoff", type="number", value=2.0, min=0.01, step=0.1, style={"width":"100%"}),
                ]),
                html.Div([
                    html.Label("BW order"),
                    dcc.Input(id="in-bw-order", type="number", value=4, min=1, step=1, style={"width":"100%"}),
                ]),
            ]
        ),

        html.Div(
            style={"marginTop": "6px", "fontSize": "12px", "opacity": 0.75},
            children=("Tip: Window must be odd. For SG, window ≥ polyorder+2. "
                      "For Butterworth, cutoff must be < Nyquist (fs/2).")
        ),

        dcc.Graph(id="graph", figure=go.Figure(), style={"height": "70vh", "marginTop": "8px"}),

        # Stores for preserving UI state
        dcc.Store(id="xrange-store"),
        dcc.Store(id="current-df-meta"),
    ]
)

# ---------- Callbacks ----------
@app.callback(
    Output("dd-sensor", "options"),
    Output("dd-sensor", "value"),
    Input("dd-action", "value"),
)
def on_action_change(action):
    if not action:
        return [], None
    sensors = list_sensors_for_action(data, action)
    opts = [{"label": s, "value": s} for s in sensors]
    val = sensors[0] if sensors else None
    return opts, val

@app.callback(
    Output("dd-rec", "options"),
    Output("dd-rec", "value"),
    Input("dd-action", "value"),
    Input("dd-sensor", "value"),
)
def on_sensor_change(action, sensor):
    if not action or not sensor:
        return [], None
    recs = list_recording_indices(data, action, sensor)
    opts = [{"label": str(i), "value": i} for i in recs]
    val = recs[0] if recs else None
    return opts, val

@app.callback(
    Output("cl-axes", "options"),
    Output("cl-axes", "value"),
    Output("current-df-meta", "data"),
    Input("dd-action", "value"),
    Input("dd-sensor", "value"),
    Input("dd-rec", "value"),
)
def update_axes(action, sensor, idx):
    if not action or not sensor or idx is None:
        return [], [], None
    try:
        df = data[action][sensor][idx]
        if df is None or df.empty:
            return [], [], None
        axes = available_axes(df)
        opts = [{"label": a.upper(), "value": a} for a in axes]
        return opts, axes, {"action": action, "sensor": sensor, "idx": idx}
    except Exception:
        return [], [], None

# Remember zoom/pan; keep when changing controls
@app.callback(
    Output("xrange-store", "data"),
    Input("graph", "relayoutData"),
    State("xrange-store", "data"),
    prevent_initial_call=True,
)
def remember_zoom(relayout, stored):
    if not relayout:
        return no_update
    if relayout.get("xaxis.autorange"):
        return None
    x0 = relayout.get("xaxis.range[0]") or (relayout.get("xaxis.range", [None, None])[0] if "xaxis.range" in relayout else None)
    x1 = relayout.get("xaxis.range[1]") or (relayout.get("xaxis.range", [None, None])[1] if "xaxis.range" in relayout else None)
    if x0 is None or x1 is None:
        return no_update
    try:
        return {"x0": float(x0), "x1": float(x1)}
    except Exception:
        return no_update

@app.callback(
    Output("graph", "figure"),
    Input("current-df-meta", "data"),
    Input("cl-axes", "value"),
    Input("sl-downsample", "value"),
    Input("ri-showmode", "value"),
    Input("dd-smooth", "value"),
    Input("in-ma-sg-window", "value"),
    Input("in-sg-poly", "value"),
    Input("in-bw-cutoff", "value"),
    Input("in-bw-order", "value"),
    State("xrange-store", "data"),
)
def update_graph(meta, axes_selected, ds, showmode,
                 sm_method, ma_sg_window, sg_poly, bw_cutoff, bw_order,
                 xr):
    if not meta:
        return go.Figure()

    action = meta["action"]; sensor = meta["sensor"]; idx = meta["idx"]
    df = data[action][sensor][idx]
    if df is None or df.empty or "seconds_elapsed" not in df.columns:
        return go.Figure()

    df = df.sort_values("seconds_elapsed", kind="stable").dropna()

    # Downsample first (so smoothing runs on what you plot)
    if isinstance(ds, int) and ds > 1:
        df = df.iloc[::ds, :]

    axes_all = available_axes(df)
    if not axes_selected:
        axes_selected = axes_all

    # Build smoothing params
    params = {
        "replace": (showmode == "sm"),
        "ma_window": int(ma_sg_window or 5),
        "sg_window": int(ma_sg_window or 7),
        "sg_poly": int(sg_poly or 3),
        "bw_cutoff": float(bw_cutoff or 2.0),
        "bw_order": int(bw_order or 4),
    }

    # Apply smoothing (adds *_sm columns if not replacing)
    df_sm = apply_smoothing(df, axes_selected, sm_method or "none", params)

    fig = go.Figure()

    # Decide which traces to draw
    draw_raw = showmode in ("raw", "overlay")
    draw_sm  = (showmode in ("sm", "overlay")) and (sm_method != "none")

    for a in axes_selected:
        if draw_raw and a in df_sm.columns:
            fig.add_trace(go.Scatter(
                x=df_sm["seconds_elapsed"], y=df_sm[a],
                mode="lines", name=a.upper()+" (raw)",
                line=dict(dash="dot"),
                hovertemplate="t=%{x:.3f}s<br>%{y:.5f}<extra>"+a.upper()+" raw</extra>"
            ))
        if draw_sm:
            sm_col = a if params["replace"] else f"{a}_sm"
            if sm_col in df_sm.columns:
                fig.add_trace(go.Scatter(
                    x=df_sm["seconds_elapsed"], y=df_sm[sm_col],
                    mode="lines", name=a.upper()+" (smoothed)",
                    hovertemplate="t=%{x:.3f}s<br>%{y:.5f}<extra>"+a.upper()+" sm</extra>"
                ))

    # If smoothing is 'none' but user chose "Smoothed only", fall back to raw
    if sm_method == "none" and showmode == "sm":
        for a in axes_selected:
            if a in df_sm.columns:
                fig.add_trace(go.Scatter(
                    x=df_sm["seconds_elapsed"], y=df_sm[a],
                    mode="lines", name=a.upper(),
                ))

    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="Value",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
        margin=dict(l=40, r=20, t=30, b=40),
        uirevision="keep-zoom",  # preserve zoom/pan across updates
    )
    fig.update_xaxes(rangeslider=dict(visible=True))

    # Re-apply stored range if present
    if xr and all(k in xr for k in ("x0", "x1")):
        fig.update_xaxes(range=[xr["x0"], xr["x1"]])

    return fig

# ---------- Launch (works in Jupyter) ----------
PORT = 8050
app.run(host="127.0.0.1", port=PORT, debug=False)


## Create Samples

After preparing and pre-processing the data, the dataset now must be split into samples, i.e. small timeframes of the total dataset. This will allow us to later assign a number of these to a training set, while the rest will be used for testing.

The function presented below will be used later to extract these samples from the data.

In [ ]:
def _split_into_samples(data: pd.DataFrame, window_s: float = 5.0) -> List[pd.DataFrame]:
    
    # Identify end time (last timestamp in data recording)
    t_end = float(data["seconds_elapsed"].iloc[-1])

    out: List[pd.DataFrame] = []

    # Call start (time = 0s)
    start = 0.0

    # Loop until end of recording
    while start + window_s <= t_end + 1e-9:

        # End of current sample
        end = start + window_s

        # Extract the rows in the current sample window
        mask = (data["seconds_elapsed"] >= start) & (data["seconds_elapsed"] < end)
        chunk = data.loc[mask].copy()

        # Reset start time to t = 0s
        chunk["seconds_elapsed"] = chunk["seconds_elapsed"] - chunk["seconds_elapsed"].iloc[0]
        out.append(chunk.reset_index(drop=True))

        # Continue to next sample window
        start = end
    return out

Similarly to some earlier functions in this notebook, the function above must be applied to each axis, of each sensor, for each action. To do this, the function below again loops over these to make sure that this happens for each instance.

In [ ]:
def build_samples(data: Dict[str, Dict[str, List[pd.DataFrame]]]) -> Dict[str, Dict[str, List[pd.DataFrame]]]:
    """
    Convert data into samples:
        samples[action][sensor] = list of 5s DataFrames
    """
    samples: Dict[str, Dict[str, List[pd.DataFrame]]] = {}
    for action, sensors in data.items():
        if action != 'sittingDown+StandingUp':
            samples[action] = {}
        else:
            samples['sittingDown'] = {}
            samples['standingUp'] = {}
        for sensor, recordings in sensors.items():
            if action != 'sittingDown+StandingUp':
                sensor_samples: List[pd.DataFrame] = []
                for rec_df in recordings:
                    split_samples = _split_into_samples(rec_df)
                    sensor_samples.extend(split_samples)
                samples[action][sensor] = sensor_samples
            else:
                sitting_down_samples: List[pd.DataFrame] = []
                standing_up_samples: List[pd.DataFrame] = []
                for rec_df in recordings:
                    split_samples = _split_into_samples(rec_df)
                    sitting_down_samples.extend(split_samples[0::2])
                    standing_up_samples.extend(split_samples[1::2])
                samples['sittingDown'][sensor] = sitting_down_samples
                samples['standingUp'][sensor] = standing_up_samples
    return samples

samples = build_samples(data)


## Split Data

With the data now extracted into samples, the total sample database can be split into a training and testing dataset.

In [257]:
def split_samples(
    samples: Dict[str, Dict[str, List[pd.DataFrame]]],
    train_ratio: float,
    seed: int = 42,
) -> Tuple[Dict[str, Dict[str, List[pd.DataFrame]]],
           Dict[str, Dict[str, List[pd.DataFrame]]],
           Dict[str, Dict[str, List[pd.DataFrame]]]]:
    
    def _init_empty_object(src):
        return {action: {sensor: [] for sensor in sensors}
            for action, sensors in src.items()}
    
    train = _init_empty_object(samples)
    test  = _init_empty_object(samples)
    
    random_number = random.Random(seed)

    for action in samples.keys():
        for sensor in samples[action].keys():
            sample_list = samples[action][sensor] 
            n = len(sample_list)

            sample_nums = list(range(n))
            random_number.shuffle(sample_nums)

            # Compute split sizes (ensure they sum to n)
            n_train = int(n * train_ratio)

            sample_nums_train = sample_nums[:n_train]
            sample_nums_test  = sample_nums[n_train:]


            train[action][sensor] = [sample_list[i] for i in sample_nums_train]
            test[action][sensor]  = [sample_list[i] for i in sample_nums_test]

    return train, test

train_samples, test_samples = split_samples(samples, 0.80)

## Visualize Samples

In [247]:
# app_samples_viewer.py
import os
from typing import Dict, List
import pandas as pd
from dash import Dash, dcc, html, Input, Output, State, ctx, no_update
import plotly.graph_objs as go

# ---------- IMPORTANT ----------
# Expect these to be available in the global scope if you've split already:
#   train_samples, val_samples, test_samples
# If not present, the app will fall back to `samples`.
try:
    samples  # type: ignore # noqa: F821
except NameError:
    # Dummy fallback to avoid NameError if you run this file standalone.
    # Replace this with your real data before running.
    samples = {"walking": {"Accelerometer": []}}

# Helper: return the dict for a chosen dataset name
def get_dataset_dict(dataset_name: str) -> Dict[str, Dict[str, List[pd.DataFrame]]]:
    # Prefer explicitly split sets if available; otherwise fallback to `samples`
    if dataset_name == "train" and "train_samples" in globals():
        return globals()["train_samples"]
    if dataset_name == "val" and "val_samples" in globals():
        return globals()["val_samples"]
    if dataset_name == "test" and "test_samples" in globals():
        return globals()["test_samples"]
    # Fallback (treat whole thing as "all")
    return samples

def get_actions(dataset_name: str) -> List[str]:
    ds = get_dataset_dict(dataset_name)
    return list(ds.keys())

def get_sensors(dataset_name: str, action: str) -> List[str]:
    ds = get_dataset_dict(dataset_name)
    return list(ds.get(action, {}).keys())

def get_sample_count(dataset_name: str, action: str, sensor: str) -> int:
    ds = get_dataset_dict(dataset_name)
    return len(ds.get(action, {}).get(sensor, []))

def make_sample_options(dataset_name: str, action: str, sensor: str, max_options: int = 10000):
    n = get_sample_count(dataset_name, action, sensor)
    return [{"label": f"Sample {i}", "value": i} for i in range(min(n, max_options))]

app = Dash(__name__)
server = app.server

app.layout = html.Div([
    html.H2("Sample Browser"),
    html.Div([
        html.Div([
            html.Label("Dataset"),
            dcc.Dropdown(
                id="ddl-dataset",
                options=[
                    {"label": "Train", "value": "train"},
                    {"label": "Validation", "value": "val"},
                    {"label": "Test", "value": "test"},
                    {"label": "All (fallback to samples)", "value": "all"},
                ],
                value="train" if "train_samples" in globals() else ("all"),
                clearable=False,
            ),
        ], style={"width": "18%"}),

        html.Div([
            html.Label("Action"),
            dcc.Dropdown(
                id="ddl-action",
                options=[],
                value=None,
                clearable=False,
            ),
        ], style={"width": "22%"}),

        html.Div([
            html.Label("Sensor"),
            dcc.Dropdown(
                id="ddl-sensor",
                options=[],
                value=None,
                clearable=False,
            ),
        ], style={"width": "22%"}),

        html.Div([
            html.Label("Samples"),
            dcc.Dropdown(
                id="ddl-samples",
                options=[],
                value=[],
                multi=True,
                placeholder="Select one or more samples",
            ),
        ], style={"width": "28%"}),

        html.Div([
            html.Label("View Mode"),
            dcc.RadioItems(
                id="rad-mode",
                options=[
                    {"label": "Overlay", "value": "overlay"},
                    {"label": "Separate", "value": "separate"},
                ],
                value="overlay",
                inline=True,
            ),
        ], style={"width": "10%"}),
    ], style={"display": "flex", "gap": "1rem", "flexWrap": "wrap", "alignItems": "end"}),

    html.Div([
        dcc.Checklist(
            id="chk-axes",
            options=[{"label": "x", "value": "x"},
                     {"label": "y", "value": "y"},
                     {"label": "z", "value": "z"}],
            value=["x", "y", "z"],
            inline=True
        ),
        html.Span("  (toggle axes)", style={"marginLeft": "0.5rem", "color": "#666"})
    ], style={"margin": "0.5rem 0 1rem 0"}),

    dcc.Loading(
        dcc.Graph(id="graph", figure=go.Figure()),
        type="default"
    ),
], style={"maxWidth": "1200px", "margin": "1.5rem auto", "fontFamily": "sans-serif"})

# --- Callbacks ---

# Populate actions based on dataset, and persist selection if valid
@app.callback(
    Output("ddl-action", "options"),
    Output("ddl-action", "value"),
    Input("ddl-dataset", "value"),
    State("ddl-action", "value"),
)
def on_dataset_change(dataset_name, current_action):
    actions = get_actions(dataset_name or "all")
    opts = [{"label": a, "value": a} for a in actions]
    if not actions:
        return [], None
    value = current_action if current_action in actions else actions[0]
    return opts, value

# Keep sensor selection when action or dataset changes (if still valid)
@app.callback(
    Output("ddl-sensor", "options"),
    Output("ddl-sensor", "value"),
    Input("ddl-dataset", "value"),
    Input("ddl-action", "value"),
    State("ddl-sensor", "value"),
)
def on_action_change(dataset_name, action, current_sensor):
    if not dataset_name or not action:
        return [], None
    sensors = get_sensors(dataset_name, action)
    opts = [{"label": s, "value": s} for s in sensors]
    if not sensors:
        return [], None
    value = current_sensor if current_sensor in sensors else sensors[0]
    return opts, value

# Keep sample selection when sensor/action/dataset changes (if still valid)
@app.callback(
    Output("ddl-samples", "options"),
    Output("ddl-samples", "value"),
    Input("ddl-dataset", "value"),
    Input("ddl-action", "value"),
    Input("ddl-sensor", "value"),
    State("ddl-samples", "value"),
)
def on_sensor_change(dataset_name, action, sensor, current_samples):
    if not dataset_name or not action or not sensor:
        return [], []
    opts = make_sample_options(dataset_name, action, sensor)
    valid = {o["value"] for o in opts}
    kept = [v for v in (current_samples or []) if v in valid]
    # auto-pick first if nothing selected
    if not kept and opts:
        kept = [opts[0]["value"]]
    return opts, kept

@app.callback(
    Output("graph", "figure"),
    Input("ddl-dataset", "value"),
    Input("ddl-action", "value"),
    Input("ddl-sensor", "value"),
    Input("ddl-samples", "value"),
    Input("rad-mode", "value"),
    Input("chk-axes", "value"),
)
def update_plot(dataset_name, action, sensor, selected_samples, mode, axes_on):
    fig = go.Figure()

    if not dataset_name or not action or not sensor or not selected_samples:
        fig.update_layout(
            title="Select dataset, action, sensor, and samples",
            xaxis_title="Time (s)",
            yaxis_title="Value",
            template="plotly_white",
            height=550,
        )
        return fig

    ds = get_dataset_dict(dataset_name if dataset_name != "all" else "all")
    rec_samples = ds.get(action, {}).get(sensor, [])

    def traces_for(df, prefix):
        traces = []
        for axis in ["x", "y", "z"]:
            if axis in axes_on and axis in df.columns:
                traces.append(go.Scatter(
                    x=df["seconds_elapsed"],
                    y=df[axis],
                    mode="lines",
                    name=f"{prefix}{axis}",
                    hovertemplate="t=%{x:.3f}s<br>"+f"{axis}=%{{y:.3f}}<extra></extra>"
                ))
        return traces

    if mode == "overlay":
        for idx in selected_samples:
            if 0 <= idx < len(rec_samples):
                df = rec_samples[idx]
                fig.add_traces(traces_for(df, prefix=f"#{idx} "))
        fig.update_layout(
            title=f"[{dataset_name}] {action} — {sensor} — overlay {len(selected_samples)} sample(s)",
            xaxis_title="Time (s)",
            yaxis_title="Value",
            template="plotly_white",
            height=650,
            legend={"orientation": "h", "yanchor": "bottom", "y": 1.02, "x": 0.01},
        )
    else:
        rows = len(selected_samples)
        domains = []
        gap = 0.04
        panel = (1.0 - (rows - 1) * gap) / rows
        for r in range(rows):
            top = 1.0 - r * (panel + gap)
            bottom = top - panel
            domains.append((bottom, top))

        for r, idx in enumerate(selected_samples):
            if 0 <= idx < len(rec_samples):
                df = rec_samples[idx]
                yaxis_name = "yaxis" if r == 0 else f"yaxis{r+1}"
                for tr in traces_for(df, prefix=f"#{idx} "):
                    tr.update(yaxis=f"y{'' if r == 0 else r+1}")
                    fig.add_trace(tr)
                fig.update_layout(**{
                    yaxis_name: dict(domain=list(domains[r]), title=f"Sample #{idx}"),
                })

        fig.update_layout(
            title=f"[{dataset_name}] {action} — {sensor} — {rows} separate panel(s)",
            xaxis=dict(title="Time (s)"),
            template="plotly_white",
            height=max(350, 280 * rows),
            legend={"orientation": "h", "yanchor": "bottom", "y": 1.02, "x": 0.01},
            margin=dict(t=60, r=30, l=60, b=40)
        )

    return fig

if __name__ == "__main__":
    # For Dash >= 2.0 use app.run, not app.run_server
    host = os.environ.get("HOST", "127.0.0.1")
    port = int(os.environ.get("PORT", "8050"))
    app.run(host=host, port=port, debug=False)


## Prepare for training models

In [255]:
# --- Toggle to turn fourier transformation on/off ---
USE_FFT = True            # set to False to use simple time-domain means
FFT_TARGET_LEN = 256      # resample each signal to this many points per 5s window
FFT_N_KEEP     = 32       # number of low-frequency rFFT bins to keep (including DC)

# --- Imports & utils ---



_TIME_COL = "seconds_elapsed"

def _resample_to_fixed_length(t: np.ndarray, x: np.ndarray, n: int) -> np.ndarray:
    """Linear interpolation to n points on [t0, t1]. Assumes t is increasing."""
    if len(x) == 1:
        return np.repeat(x.astype(float), n)
    t0, t1 = float(t[0]), float(t[-1])
    tgt = np.linspace(t0, t1, num=n, endpoint=True)
    return np.interp(tgt, t.astype(float), x.astype(float))

def _trial_features_mean(df: pd.DataFrame) -> pd.Series:
    """Time-domain aggregation (column means, excluding time)."""
    cols = [c for c in df.columns if c != _TIME_COL]
    if df.shape[0] == 1:
        s = df[cols].iloc[0]
    else:
        s = df[cols].mean(axis=0)
    s.index = pd.Index([f"{c}" for c in s.index])
    return s

def _trial_features_fft(
    df: pd.DataFrame,
    target_len: int = FFT_TARGET_LEN,
    n_keep: int = FFT_N_KEEP
) -> pd.Series:
    """
    rFFT magnitude features per channel after resampling each channel to a fixed length.
    Returns first n_keep bins (including DC). Columns that are non-numeric are skipped.
    """
    assert _TIME_COL in df.columns, f"Expected time column '{_TIME_COL}'."
    t = df[_TIME_COL].to_numpy()
    out_parts = []
    for col in df.columns:
        if col == _TIME_COL:
            continue
        x = pd.to_numeric(df[col], errors="coerce").to_numpy()
        if np.all(np.isnan(x)):
            continue
        x = np.nan_to_num(x, nan=np.nanmean(x) if np.isfinite(np.nanmean(x)) else 0.0)
        xr = _resample_to_fixed_length(t, x, target_len)
        # Real FFT, normalize by length for scale invariance
        spec = np.fft.rfft(xr)
        mag = np.abs(spec) / target_len
        k = min(n_keep, mag.shape[0])
        vals = mag[:k]
        idx = pd.Index([f"{col}__fft{k_}" for k_ in range(k)])
        out_parts.append(pd.Series(vals, index=idx))
    if not out_parts:
        # fallback to zeros if all channels invalid
        return pd.Series(dtype=float)
    return pd.concat(out_parts)

def _aggregate_trial_df(df: pd.DataFrame) -> pd.Series:
    """Select featureization mode."""
    return _trial_features_fft(df) if USE_FFT else _trial_features_mean(df)

def _concat_sensors_for_trial(trial_per_sensor: Dict[str, pd.DataFrame]) -> pd.Series:
    parts = []
    for sensor, df in trial_per_sensor.items():
        s = _aggregate_trial_df(df)
        # Prefix with sensor name
        s.index = pd.Index([f"{sensor}__{c}" for c in s.index])
        parts.append(s)
    return pd.concat(parts) if parts else pd.Series(dtype=float)

def build_Xy(split_dict: Dict[str, Dict[str, List[pd.DataFrame]]]) -> Tuple[pd.DataFrame, np.ndarray, List[str]]:
    rows, labels = [], []
    for activity, sensors in split_dict.items():
        first_sensor = next(iter(sensors))
        n_trials = len(sensors[first_sensor])
        for sensor, lst in sensors.items():
            if len(lst) != n_trials:
                raise ValueError(f"Inconsistent trials for {activity}: {first_sensor}={n_trials}, {sensor}={len(lst)}")
        for j in range(n_trials):
            trial_per_sensor = {s: sensors[s][j] for s in sensors.keys()}
            rows.append(_concat_sensors_for_trial(trial_per_sensor))
            labels.append(activity)
    # Align columns across variable-length Series by outer join
    X = pd.DataFrame(rows).fillna(0.0).reset_index(drop=True)
    le = LabelEncoder()
    y = le.fit_transform(labels)
    classes = list(le.classes_)
    return X, y, classes

def evaluate(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    return {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "kappa": float(cohen_kappa_score(y_true, y_pred)),
        "f1_macro": float(f1_score(y_true, y_pred, average="macro")),
    }

def align_clusters(y_true: np.ndarray, y_cluster: np.ndarray, n_classes: int) -> Tuple[np.ndarray, Dict[int, int]]:
    C = confusion_matrix(y_true, y_cluster, labels=np.arange(n_classes))
    cost = C.max() - C
    r, c = linear_sum_assignment(cost)
    mapping = {cluster: cls for cluster, cls in zip(c, r)}
    y_aligned = np.array([mapping[z] for z in y_cluster])
    return y_aligned, mapping

# --- Build matrices (now FFT-based if USE_FFT=True) ---
X_train, y_train, classes = build_Xy(train_samples)
X_val,   y_val,   _       = build_Xy(val_samples)
X_test,  y_test,  _       = build_Xy(test_samples)
n_classes = len(classes)


# Supervised models

## Logistic regression

In [256]:
logreg = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.95)),
    ("clf", LogisticRegression(max_iter=2000, random_state=42))
])

logreg.fit(X_train, y_train)
y_val_hat  = logreg.predict(X_val)
y_test_hat = logreg.predict(X_test)

logreg_val_metrics  = evaluate(y_val,  y_val_hat)
logreg_test_metrics = evaluate(y_test, y_test_hat)
logreg_val_metrics, logreg_test_metrics


({'accuracy': 0.9916666666666667,
  'kappa': 0.9895833333333334,
  'f1_macro': 0.9916630481980026},
 {'accuracy': 0.9939393939393939,
  'kappa': 0.9924242424242424,
  'f1_macro': 0.9939380022962112})

## KNN

In [245]:
knn = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.95)),
    ("clf", KNeighborsClassifier(n_neighbors=5))
])

knn.fit(X_train, y_train)
y_val_hat  = knn.predict(X_val)
y_test_hat = knn.predict(X_test)

knn_val_metrics  = evaluate(y_val,  y_val_hat)
knn_test_metrics = evaluate(y_test, y_test_hat)
knn_val_metrics, knn_test_metrics

({'accuracy': 0.9666666666666667,
  'kappa': 0.9583333333333334,
  'f1_macro': 0.9664857432334635},
 {'accuracy': 0.984, 'kappa': 0.98, 'f1_macro': 0.984})

## Gaussian Naive Bayes

In [231]:
gnb = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.95)),
    ("clf", GaussianNB())
])

gnb.fit(X_train, y_train)
y_val_hat  = gnb.predict(X_val)
y_test_hat = gnb.predict(X_test)

gnb_val_metrics  = evaluate(y_val,  y_val_hat)
gnb_test_metrics = evaluate(y_test, y_test_hat)
gnb_val_metrics, gnb_test_metrics

({'accuracy': 1.0, 'kappa': 1.0, 'f1_macro': 1.0},
 {'accuracy': 1.0, 'kappa': 1.0, 'f1_macro': 1.0})

## Decision Tree

In [232]:
dt_pipe = Pipeline([
    ("pca", PCA(n_components=0.95)),
    ("clf", DecisionTreeClassifier(random_state=42))
])

# Fit the pipeline
dt_pipe.fit(X_train, y_train)

y_val_hat  = dt_pipe.predict(X_val)
y_test_hat = dt_pipe.predict(X_test)

dt_val_metrics  = evaluate(y_val,  y_val_hat)
dt_test_metrics = evaluate(y_test, y_test_hat)
dt_val_metrics, dt_test_metrics

({'accuracy': 1.0, 'kappa': 1.0, 'f1_macro': 1.0},
 {'accuracy': 0.9555555555555556,
  'kappa': 0.9444444444444444,
  'f1_macro': 0.9564705882352941})

# Unsupervised models

## k-Means

In [210]:
kmeans = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.95)),
    ("kmeans", KMeans(n_clusters=n_classes, n_init="auto", random_state=42))
])

kmeans.fit(X_train)
k_train = kmeans[-1].labels_
k_val   = kmeans.predict(X_val)  # uses scaler + kmeans

y_train_aligned, map_k = align_clusters(y_train, k_train, n_classes)
y_val_aligned = np.array([map_k[z] for z in k_val])

kmeans_val_metrics = evaluate(y_val, y_val_aligned)
kmeans_val_metrics

{'accuracy': 0.7833333333333333,
 'kappa': 0.7291666666666667,
 'f1_macro': 0.7663945578231293}

## Gaussian Mixture Model

In [173]:
gmm_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.3)),
    ("gmm", GaussianMixture(n_components=n_classes, 
                            covariance_type="full", 
                            random_state=42))
])

gmm_pipe.fit(X_train)
g_train = gmm_pipe.predict(X_train)
g_val   = gmm_pipe.predict(X_val)



y_train_aligned, map_g = align_clusters(y_train, g_train, n_classes)
y_val_aligned = np.array([map_g[z] for z in g_val])

gmm_val_metrics = evaluate(y_val, y_val_aligned)
gmm_val_metrics

{'accuracy': 0.6583333333333333,
 'kappa': 0.5729166666666667,
 'f1_macro': 0.6175236277581693}

## Fuzzy C-Means

In [174]:
# scale then FCM (skfuzzy expects features x samples)
scaler = StandardScaler().fit(X_train.values)
X_train_scaled = scaler.transform(X_train.values)  # d x n
X_val_scaled = scaler.transform(X_val.values)

pca = PCA(n_components=0.90)
X_train_pca = pca.fit_transform(X_train_scaled)
X_val_pca   = pca.transform(X_val_scaled)

# FCM expects features x samples
Xtr = X_train_pca.T
Xva = X_val_pca.T

cntr, u_tr, _, _, _, _, _ = fuzz.cluster.cmeans(
    data=Xtr, c=n_classes, m=2.0, error=1e-5, maxiter=1000, seed=42
)
f_train = np.argmax(u_tr, axis=0)

# align clusters
y_train_aligned, map_f = align_clusters(y_train, f_train, n_classes)

# predict on validation
u_va, _, _, _, _, _ = fuzz.cluster.cmeans_predict(
    test_data=Xva, cntr_trained=cntr, m=2.0, error=1e-8, maxiter=50
)
f_val = np.argmax(u_va, axis=0)
y_val_aligned = np.array([map_f[z] for z in f_val])

fcm_val_metrics = evaluate(y_val, y_val_aligned)
fcm_val_metrics

{'accuracy': 0.8, 'kappa': 0.75, 'f1_macro': 0.7333333333333333}